In [10]:
from netCDF4 import Dataset
import numpy as np
fin  = Dataset('f522_dh.trainingdata_in.lcv.20190401_0000z.nc4') 
fout = Dataset('f522_dh.trainingdata_out.lcv.20190401_0000z.nc4')

In [11]:
x = fin.variables
y = fout.variables
for var in x.keys():
    print(var, x[var].shape)
    
n   = x['Xdim'].shape[0]    
p   = x['Ydim'].shape[0]

Xdim (720,)
Ydim (4320,)
lons (4320, 720)
lats (4320, 720)
lev (72,)
time (1,)
emis (1, 4320, 720)
fcld (1, 72, 4320, 720)
o3 (1, 72, 4320, 720)
pl (1, 72, 4320, 720)
q (1, 72, 4320, 720)
qi (1, 72, 4320, 720)
ql (1, 72, 4320, 720)
ri (1, 72, 4320, 720)
rl (1, 72, 4320, 720)
t (1, 72, 4320, 720)
ts (1, 4320, 720)


### DATA PREPROCESSING

In [7]:
class Kernel(object):
    def __init__(self, ids):
        self.ids = ids
        
    def apply(self, x):
        return(x)       

class ProdKernel(Kernel):
    def __init__(self, ids=[]):
        Kernel.__init__(self,ids)
        self.ids=ids
        self.header=[]

    def apply(self,x0, xheader):
        x=x0.copy()
        for ids in self.ids:
            id1=np.where(ids[0]==xheader)[0]
            id2=np.where(ids[1]==xheader)[0]
            k = x[:,:,id1]*x[:,:,id2]
            xheader.append(ids[0]+'*'+ids[1])
            x = np.concatenate((k,x),axis=-1)
        return(x)

class FKernel(Kernel):
    def __init__(self, func, gamma=1, ids=[]):
        Kernel.__init__(self,ids)
        self.func = func
        self.fname = str(func).split(' ')[1]
        self.ids  = ids

    def apply(self,x0, xheader0):
        xheader0 = xheader.copy()
        self.header=[]
        x=x0.copy()
        for ids in self.ids:
            id1=np.where(ids[0]==xheader)[0]
            k = self.func(x[:,:,id1])
            xheader.append(fname+ids[0])
            x = np.concatenate((k,x),axis=-1)
        self.xheader=xheader
        return(x)

Pk = ProdKernel()
Funcf = FKernel(np.exp)

In [58]:
def GenerateConv(x, y, kernel,train_prop=0.9, header=['ql'], batch_size=16, seed=0):
    "Generate a batch, randomly for a convolution NN, using only variable in header"

    n  = x['Xdim'].shape[0]
    nt = int(n*train_prop) # id up to training
    p  = x['Ydim'].shape[0]
    lev= x['lev'].shape[0]

    xheader = header
    xheader.sort()
    
    while True:
        yshuffled   = np.arange(p) # y id but random
        xshuffled   = np.arange(nt)# x id but random
        
        #np.random.shuffle(yshuffled) #shuffling y divide the spead by 3
        np.random.shuffle(xshuffled)
        id_batches_x = 0 #counting the id for batches coordinates
        id_batches_y = 0
        
        while (id_batches_x) < nt+1:

            dataX = np.zeros((batch_size,lev,1)) # batch data
            idn   = xshuffled[id_batches_x]
            idp   = yshuffled[id_batches_y + np.arange(batch_size)]
            Y = y['flx'][0,:, idp, idn]
            Y = Y.swapaxes(0,1)
            
            for k in xheader:
                if(len(x[k].shape)==4):
                    a = x[k][:,:,idp,idn]
                    a = a.reshape(1,lev, -1)
                    a = a.swapaxes(0,2)
                elif(len(x[k].shape)==3):
                    a = x[k][:,idp,idn]
                    a = a.repeat(lev,1).reshape(1,-1 , lev)
                    a = a.swapaxes(0,1)
                    a = a.swapaxes(1,2)
                dataX = np.concatenate((dataX,a),axis=2)
            dataX = dataX[:,:,1:] # the first channel is full of 0
            kernel.apply(dataX, xheader)
            print()
            yield dataX,Y
            id_batches_y += batch_size 
            if(id_batches_y+batch_size >= p):
                id_batches_y = 0
                id_batches_x+= 1

from tqdm import tqdm
batch_size=16
generate_batch = lambda A,batch_size : GenerateConv(A[0],A[1],A[2],batch_size=16, header=["t","ts", "o3", 'emis','ql'])

for i in tqdm(range(1)):
    for X,Y in generate_batch( (x,y,Prodk),batch_size ):
        print(X.shape,Y.shape)
        break


100%|██████████| 1/1 [00:00<00:00,  1.93it/s]


(16, 72, 5) (16, 73)


In [57]:
x.keys()

odict_keys(['Xdim', 'Ydim', 'lons', 'lats', 'lev', 'time', 'emis', 'fcld', 'o3', 'pl', 'q', 'qi', 'ql', 'ri', 'rl', 't', 'ts'])

In [54]:
def GenerateDense(x, y, kernel, train_prop=0.9, header=["o3",'ql',"t","ts",'emis'], batch_size=16, Dense=False, seed=0):
    "Generate a batch, randomly for a dense NN, using only variable in header"
    n  = x['Xdim'].shape[0]
    nt = int(n*train_prop) # id up to training
    p  = x['Ydim'].shape[0]
    lev= x['lev'].shape[0]

    np.random.seed(seed)
    xheader = header
    xheader.sort()
    
    while True:
        yshuffled   = np.arange(p) # y id but random
        xshuffled   = np.arange(nt)# x id but random
        
        #np.random.shuffle(yshuffled) #shuffling y divide the spead by 3
        np.random.shuffle(xshuffled)
        id_batches_x = 0 #counting the id for batches coordinates
        id_batches_y = 0
        
        while (id_batches_x) < nt+1:

            dataX = np.zeros((batch_size, 1)) # batch data
            idn   = xshuffled[id_batches_x]
            idp   = yshuffled[id_batches_y + np.arange(batch_size)]
            Y = y['flx'][0,:, idp, idn]
            Y = Y.swapaxes(0,1)
            for k in xheader:
                if(len(x[k].shape)==4):
                    a = x[k][:,:,idp,idn]
                    a = a.reshape(lev, -1)
                    a = a.swapaxes(0,1)
                elif(len(x[k].shape)==3):
                    a = x[k][:,idp,idn]
                    a = a.swapaxes(0,1)
                dataX = np.concatenate((dataX,a),axis=1)
                
            dataX = dataX[:,1:] # the first column is full of zeros
            kernel.apply(dataX, xheader)
            yield dataX,Y
            id_batches_y += batch_size 
            if(id_batches_y+batch_size >= p):
                id_batches_y = 0
                id_batches_x+= 1

batch_size=16
generate_batch = lambda A,batch_size : GenerateDense(A[0],A[1],A[2],batch_size=16)

for i in tqdm(range(1)):
    for X,Y in generate_batch( (x,y,Prodk),batch_size ):
        print(X.shape,Y.shape)
        break


100%|██████████| 1/1 [00:00<00:00,  1.99it/s]

(16, 218) (16, 73)


In [28]:
for i in tqdm(range(10)):
    yshuffled   = np.arange(p) # y id but random
    xshuffled   = np.arange(n) # x id but random
    np.random.shuffle(yshuffled)
    np.random.shuffle(xshuffled)


100%|██████████| 10/10 [00:00<00:00, 5874.38it/s]

CAN BE MODIFY :
   - tride : check
   - padding : change value of padding (same for ex or lin)
   - check act/kernel constraint (limit size of gradient)

In [60]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Activation,Input,Flatten, UpSampling1D
from keras import optimizers, regularizers, initializers

# PARAMETERS 
HNn = [128,64,16,4] # hidden layer size
d=12 # number of variables ############" CHANGE !!
s=0

model = Sequential([
    Conv1D(input_shape=(72,d),filters=HNn[0],kernel_regularizer=regularizers.l2(0.01), \
           kernel_size=10, strides=1, padding="same", kernel_initializer = initializers.Zeros()),
    Activation("relu"),
    Conv1D(filters=HNn[1], kernel_size=10, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Conv1D(filters=HNn[2], kernel_size=3, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Conv1D(filters=HNn[3], kernel_size=3, \
           kernel_regularizer=regularizers.l2(0.01), strides=1, padding="same",kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s)),
    Activation("relu"),
    Flatten(),
    Dense(73, input_shape=(72*HNn[2],), kernel_regularizer = regularizers.l2(0.01),kernel_initializer = initializers.RandomUniform(minval=-1, maxval=1, seed=s))
])

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9)
model.compile(optimizer=sgd,
             loss='mse',
             )
#model.summary()
model.save('arch1_FCNN.h5')

In [ ]:
Bi

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 72, 128)           15488     
_________________________________________________________________
activation_1 (Activation)    (None, 72, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72, 64)            81984     
_________________________________________________________________
activation_2 (Activation)    (None, 72, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 72, 16)            3088      
_________________________________________________________________
activation_3 (Activation)    (None, 72, 16)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 72, 4)             196       
__________

### MODEL 2 :

- FCNN
- (with AE)

In [34]:
model2 = Sequential([
    Dense(128, input_shape=(72*(d-5), )),
    Dense(73, )
])
model2.compile(optimizer=sgd,
             loss='mse',
             )
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               64640     
_________________________________________________________________
dense_3 (Dense)              (None, 73)                9417      
Total params: 74,057
Trainable params: 74,057
Non-trainable params: 0
_________________________________________________________________


In [22]:
72*(d-4)

576

In [15]:
128*d+128

1664

In [81]:
def generator_model():  # CDNN Model
    INPUT_LN = 72
    N_GEN_l = [3,2]
    CODE_LN = 73
    print(INPUT_LN, N_GEN_l, CODE_LN) 
    
    model = Sequential()

    model.add(Conv1D(16, 5, padding='same', input_shape=(CODE_LN, 1)))
    model.add(Activation('relu'))

    model.add(UpSampling1D(size=N_GEN_l[0]))
    model.add(Conv1D(32, 5, padding='same'))
    model.add(Activation('relu'))

    model.add(UpSampling1D(size=N_GEN_l[1]))
    model.add(Conv1D(1, 5, padding='same'))
    model.add(Activation('tanh'))
    return model

m=generator_model()
m.summary()

72 [3, 2] 73
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 73, 16)            96        
_________________________________________________________________
activation_37 (Activation)   (None, 73, 16)            0         
_________________________________________________________________
up_sampling1d_21 (UpSampling (None, 219, 16)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 219, 32)           2592      
_________________________________________________________________
activation_38 (Activation)   (None, 219, 32)           0         
_________________________________________________________________
up_sampling1d_22 (UpSampling (None, 438, 32)           0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 438, 1)            161     

In [ ]:
w=model.weights[0]

##### MODEL 2 : FCM-Final FC

##### MODEL 2 : U-net :
- use regular U-net so all layers affect each other and more stability

### MODEL 3 : Bidir-LSTM 
> Possible alternatives

- use two LSTM to show both impact of superior and inferior layer
- use attention model over it
- use w embeddings before

> TD

- Read git trez
- Read article of Hedge fun

In [28]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN, SimpleRNN, GRU, Activation
from keras import optimizers
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True),input_shape=(72, 12)))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 72, 256)           144384    
_________________________________________________________________
dense_44 (Dense)             (None, 72, 50)            12850     
_________________________________________________________________
activation_29 (Activation)   (None, 72, 50)            0         
_________________________________________________________________
dense_45 (Dense)             (None, 72, 20)            1020      
_________________________________________________________________
activation_30 (Activation)   (None, 72, 20)            0         
_________________________________________________________________
dense_46 (Dense)             (None, 72, 1)             21        
Total params: 158,275
Trainable params: 158,275
Non-trainable params: 0
_________________________________________________________________


27

#### TRASH

In [8]:
def generate_batch(x, y, batch_size=16):

    n   = x['Xdim'].shape[0]
    p   = x['Ydim'].shape[0]
    lev = x['lev'].shape[0]
    idx = np.random.choice(np.arange(n*p), batch_size, replace=False)
    
    Y = y['flx'][:].reshape(1, lev, -1).swapaxes(0,2)[idx]
    
    databoard = np.zeros((batch_size,lev,1))
    xheader = []

    for k in x.keys():
        a = x[k][:]
        if(len(a.shape)>2):
            xheader.append(k)
            if(len(x[k].shape)==4):
                a = a.reshape(1,lev, -1)
                a = a.swapaxes(0,2)
            elif(len(x[k].shape)==3):
                a = a.reshape(1, -1)
                a = a.repeat(lev,1).reshape(1,-1 , lev)
                a = a.swapaxes(0,1)
                a = a.swapaxes(1,2)
            a = a[idx]
            databoard = np.concatenate((databoard,a),axis=2)
    return(databoard,xheader,Y)

X,hx,Y = generate_batch(x,y)